<a href="https://colab.research.google.com/github/gsilver321/project_gss/blob/main/project1_paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 GSS Paper
### DS 3001: Foundations of Machine Learning
### Gabe Silverstein, Rohan Chowla, Evan Stewart, and Rithwik Raman

## Summary

A one paragraph description of the question, methods, and results (about 350 words)

## Data

One to two pages discussing the data and key variables in the analysis, and any challenges in reading, cleaning, and preparing them for analysis

## Results

Two to five pages providing visualizations, statistics, and a discussion of your findings. If you have a lot of plots or tables, that’s OK, but try to focus on a few key pieces of evidence rather than doing every single pairwise comparison of some set of variables

## Conclusion

One to two pages summarizing the project, defending it from criticism, and suggesting additional work that was outside the scope of the project